In [1]:
import pandas as pd
import gurobipy as gp
import numpy as np
from gurobipy import *
from geopy.distance import geodesic
import math

In [2]:
# read csv files
#zipcode = pd.read_csv('allegheny_zipcodes.csv')
#pods = pd.read_csv('Allegheny_County_POD_Sites.csv')

# dataframe for zipcode wise population
population = pd.read_csv('allegheny_population.csv')

# dataframe for distance between zipcode centroids and POD sites
distance = pd.read_csv('distance.csv').iloc[:,1:]
distance.columns = [i for i in range(distance.shape[1])]

# dataframe for list of zip codes with outbreak
# outbreak = pd.read_csv('outbreak.csv')

In [3]:
num_areas = range(distance.shape[0])
num_pods = range(distance.shape[1])

#check: distance.shape

In [4]:
# Requirements for vaccination campaign
minPercent = 0.8
sumPopulation = sum(population['population'])


#cost coefficients for the model
h1, h2, h3, h4, h5, h6, h7, h8 = 1, 500, 700, 900, 10, 50000, 70000, 90000
T1 = 1
T2 = 0
T3 = 1
max_budget = 50*10**6

"""
some good trial ranges for this dataset:
max_budget: 50 million to 15 million
maxCapacity: 0.05 to 0.025
setObjective: travel_dist or total_cost or a linear combination of both
"""

'\nsome good trial ranges for this dataset:\nmax_budget: 50 million to 15 million\nmaxCapacity: 0.05 to 0.025\nsetObjective: travel_dist or total_cost or a linear combination of both\n'

## model 1: baseline - minimizing total travel distance


In [5]:
# set up the model and running in loop

alpha_options = [0.01, 0.02, 0.04, 0.06, 0.08, 
                 0.10, 0.20, 0.30, 0.40, 0.50, 
                 0.60, 0.70, 0.80, 0.90, 0.92, 
                 0.94, 0.96, 0.98, 0.99] # weight for cost


#empty dataframe to collect the output to create CSV of the model output
model_1_result_df = pd.DataFrame()


#POD capacity as percentage of the total population:
POD_cap_percent_pop_list = [0.02, 0.025, 0.03, 0.05, 0.07]

for POD_cap_percent_pop in POD_cap_percent_pop_list:
    
    maxCapacity = sumPopulation * POD_cap_percent_pop

    cost_results = list()

    total_dist_results = list()

    for alpha in alpha_options:


        m = Model("facility")

        built = m.addVars(num_pods, vtype=GRB.BINARY) #1 if built; X_j, 47
        assign = m.addVars(num_pods, num_areas, lb = 0.0) #F_ij, 47 * 98
        capacity = m.addVars(num_pods, lb = 0.0, vtype=GRB.INTEGER)
        total_cost = m.addVar(name = "total_cost")

        travel_dist = gp.quicksum(assign[pod,area] * 
                                  distance[pod][area] * 
                                  int(population.iloc[area]['population']) for area in num_areas for pod in num_pods)

        m.setObjective((1-alpha) * travel_dist + alpha * total_cost, GRB.MINIMIZE)

        # add constraints

        # number of pods constraint
        # m.addConstr(sum(built[j] for j in num_pods) == 20)

        # no assignment, travel should appear without a POD built
        for area in num_areas:
            for pod in num_pods:
                m.addConstr(assign[pod, area] <= built[pod])

        # capacity of a pod should be zero for the locations where the POD is not built
        bigm = sumPopulation

        for pod in num_pods:
            m.addConstr(capacity[pod] <= built[pod] * bigm)


        # upper bound for assigning fraction of population to a POD:
        for area in num_areas:
            for pod in num_pods:
                m.addConstr(assign[pod, area] <= 1)


        # Capacity limit for all PODs: capacity of all PODs should be less than maxCapacity
        for pod in num_pods:
            m.addConstr(capacity[pod] <= maxCapacity)


        # Capacity constraint: population assigned to a pod should be smaller than the POD capacity
        m.addConstrs(gp.quicksum(int(population.iloc[area]['population']) * assign[pod, area] for area in num_areas)
                         <= capacity[pod] for pod in num_pods)


        # minimum coverage constraint
        m.addConstr(minPercent * sumPopulation <= gp.quicksum(capacity[pod] for pod in num_pods))


        # population coverage constraint
        for area in num_areas:
            m.addConstr(gp.quicksum(assign[pod, area] for pod in num_pods) >= 1 * minPercent)

        # cost related constraints:

        m.addConstr(gp.quicksum((h1 * capacity[pod] + h2 * T1 + h3 * T2 + h4 * T3) * built[pod] +
                                 (h5 * capacity[pod] + h6 * T1 + h7 * T2 + h8 * T3) * built[pod] for pod in num_pods) == total_cost)

        m.addConstr(total_cost <= max_budget)

        m.Params.TimeLimit = 60
        # Solve
        m.optimize()

        assign_total = {} # dict of pod: assigned population
        for pod in num_pods:
            total_pop = 0
            for area in num_areas:
                total_pop += population.iloc[area]['population'] * assign[pod,area].X
            assign_total[pod] = total_pop

        sum(assign_total.values())/sumPopulation


        # cost results:
        cost_dict = {}
        for pod in num_pods:
            a_pod = (h1 * capacity[pod].X + h2 * T1 + h3 * T2 + h4 * T3) * built[pod].X
            b_pod = (h5 * capacity[pod].X + h6 * T1 + h7 * T2 + h8 * T3) * built[pod].X
            cost_dict[pod] = a_pod + b_pod
        sum(cost_dict.values())

        # total cost
        print(f"Total cost: {total_cost.X}")
        cost_results.append(total_cost.X)

        #travel distance:
        total_dist_opt = 0 # dict of pod: assigned population
        for pod in num_pods:
            for area in num_areas:
                total_dist_opt += (assign[pod,area].X * 
                                   distance[pod][area] * 
                                   int(population.iloc[area]['population']))

        #total distance travelled
        print(f"Total distance travelled: {total_dist_opt}")
        total_dist_results.append(total_dist_opt)


    # creating dataframe for the csv output from the model

    cost_column_name = 'Cost_at_pod_cap_' + str(POD_cap_percent_pop*100) + '_percent'
    dist_column_name = 'Dist_at_pod_cap_' + str(POD_cap_percent_pop*100) + '_percent'

    model_1_result_df['Alpha'] = alpha_options
    model_1_result_df[str(cost_column_name)] = cost_results
    model_1_result_df[str(dist_column_name)] = total_dist_results




Academic license - for non-commercial use only - expires 2022-09-10
Using license file C:\Users\Ravi\gurobi.lic
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9453 rows, 4701 columns and 23266 nonzeros
Model fingerprint: 0xdbfc5d41
Model has 1 quadratic constraint
Variable types: 4607 continuous, 94 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [1e-02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.02s
Presolved: 4941 rows, 4748 columns, 19036 nonzeros
Variable types: 4606 continuous, 142 integer (47 binary)

Root relaxation: objective 1.739891e+06, 371 iterations, 0.01 seconds

    Nodes

Total cost: 16818846.000000007
Total distance travelled: 1597730.4096
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9453 rows, 4701 columns and 23266 nonzeros
Model fingerprint: 0x93812263
Model has 1 quadratic constraint
Variable types: 4607 continuous, 94 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [8e-02, 9e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.02s
Presolved: 4941 rows, 4748 columns, 19036 nonzeros
Variable types: 4606 continuous, 142 integer (47 binary)

Root relaxation: objective 2.746804e+06, 648 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective B

Total cost: 16535980.0
Total distance travelled: 1634814.7222000016
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9453 rows, 4701 columns and 23266 nonzeros
Model fingerprint: 0x7f103f1b
Model has 1 quadratic constraint
Variable types: 4607 continuous, 94 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [3e-01, 7e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.02s
Presolved: 4941 rows, 4748 columns, 19036 nonzeros
Variable types: 4606 continuous, 142 integer (47 binary)

Root relaxation: objective 5.566912e+06, 760 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bou


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8090380.08    0   35          - 8090380.08      -     -    0s
H    0     0                    9687823.9444 8090380.08  16.5%     -    0s
H    0     0                    9539512.9578 8090380.08  15.2%     -    0s
     0     0 8329991.21    0   22 9539512.96 8329991.21  12.7%     -    0s
     0     0 8437900.86    0   29 9539512.96 8437900.86  11.5%     -    0s
     0     0 8603367.61    0   25 9539512.96 8603367.61  9.81%     -    0s
H    0     0                    9494299.3309 8603367.61  9.38%     -    0s
     0     0 8953902.86    0   24 9494299.33 8953902.86  5.69%     -    0s
H    0     0                    9481374.2947 8953902.86  5.56%     -    0s
H    0     0                    9038238.6238 8953902.86  0.93%     -    0s
     0     0 9025368.79    0   22 9038238.62 9025368.79  0.14%     -    0s
     0     0 9027624.37

H   33    44                    1.197785e+07 1.1978e+07  0.00%  20.0    0s

Cutting planes:
  MIR: 57
  Flow cover: 1
  RLT: 9

Explored 43 nodes (2194 simplex iterations) in 0.65 seconds
Thread count was 12 (of 12 available processors)

Solution count 10: 1.19779e+07 1.1981e+07 1.20395e+07 ... 1.29217e+07

Optimal solution found (tolerance 1.00e-04)
Best objective 1.197785006358e+07, best bound 1.197770651011e+07, gap 0.0012%
Total cost: 16394579.999999998
Total distance travelled: 1672146.878600001
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9453 rows, 4701 columns and 23266 nonzeros
Model fingerprint: 0x0a74037b
Model has 1 quadratic constraint
Variable types: 4607 continuous, 94 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+01, 

...

     0     2 7331495.62    0   79 7438639.48 7331495.62  1.44%     -    0s
H   31    43                    7335675.5665 7333981.53  0.02%  62.7    1s
H   70    18                    7334883.6366 7334659.71  0.00%  36.4    1s
*   73    18              14    7334883.3465 7334659.71  0.00%  35.1    1s

Cutting planes:
  Gomory: 29
  MIR: 35
  Flow cover: 6
  RLT: 14
  Relax-and-lift: 5

Explored 84 nodes (5860 simplex iterations) in 1.22 seconds
Thread count was 12 (of 12 available processors)

Solution count 10: 7.33488e+06 7.33488e+06 7.33568e+06 ... 8.0384e+06

Optimal solution found (tolerance 1.00e-04)
Best objective 7.334883346500e+06, best bound 7.334659714425e+06, gap 0.0030%
Total cost: 12293980.000000002
Total distance travelled: 2375786.6930000167
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize 

Variable types: 4607 continuous, 94 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [8e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.02s
Presolved: 4941 rows, 4748 columns, 19036 nonzeros
Variable types: 4606 continuous, 142 integer (47 binary)

Root relaxation: objective 1.808172e+06, 3663 iterations, 0.17 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1808172.20    0   53          - 1808172.20      -     -    0s
H    0     0                    2.257053e+07 1808172.20  92.0%     -    0s
H    0     0                    2.201518e+07 1808172.20  91.8%     -    0s
H    0     0                    1.239308e+07 1808172.20  85.4%     -    0s
     0     0 2513156.4


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1692877.51    0   53          - 1692877.51      -     -    0s
H    0     0                    2.569907e+07 1692877.51  93.4%     -    0s
H    0     0                    1.289816e+07 1692877.51  86.9%     -    0s
     0     0 2233203.47    0   49 1.2898e+07 2233203.47  82.7%     -    0s
H    0     0                    1.153034e+07 2233203.47  80.6%     -    0s
     0     0 1.0523e+07    0   61 1.1530e+07 1.0523e+07  8.74%     -    0s
     0     0 1.0523e+07    0   59 1.1530e+07 1.0523e+07  8.74%     -    0s
     0     0 1.1437e+07    0   67 1.1530e+07 1.1437e+07  0.81%     -    0s
     0     0 1.1437e+07    0   70 1.1530e+07 1.1437e+07  0.81%     -    0s
     0     0 1.1437e+07    0   61 1.1530e+07 1.1437e+07  0.81%     -    0s
     0     0 1.1438e+07    0   82 1.1530e+07 1.1438e+07  0.80%     -    0s
     0     0 1.1438e+07

     0     0 1.1825e+07    0   64 1.1910e+07 1.1825e+07  0.72%     -    0s
     0     0 1.1825e+07    0   48 1.1910e+07 1.1825e+07  0.71%     -    0s
     0     0 1.1825e+07    0   85 1.1910e+07 1.1825e+07  0.71%     -    0s
     0     0 1.1826e+07    0   85 1.1910e+07 1.1826e+07  0.71%     -    0s
H    0     0                    1.190401e+07 1.1826e+07  0.66%     -    0s
     0     0 1.1826e+07    0   97 1.1904e+07 1.1826e+07  0.66%     -    0s
     0     0 1.1826e+07    0   91 1.1904e+07 1.1826e+07  0.66%     -    0s
H    0     0                    1.190398e+07 1.1826e+07  0.66%     -    0s
     0     0 1.1826e+07    0   88 1.1904e+07 1.1826e+07  0.66%     -    0s
     0     0 1.1826e+07    0  103 1.1904e+07 1.1826e+07  0.66%     -    0s
     0     0 1.1826e+07    0  102 1.1904e+07 1.1826e+07  0.66%     -    1s
     0     0 1.1826e+07    0  102 1.1904e+07 1.1826e+07  0.66%     -    1s
     0     2 1.1826e+07    0  102 1.1904e+07 1.1826e+07  0.66%     -    1s
H   32    38             

     0     0 1.2116e+07    0   91 1.2335e+07 1.2116e+07  1.77%     -    1s
     0     0 1.2116e+07    0   90 1.2335e+07 1.2116e+07  1.77%     -    1s
     0     0 1.2116e+07    0   90 1.2335e+07 1.2116e+07  1.77%     -    1s
H    0     0                    1.219873e+07 1.2116e+07  0.68%     -    1s
     0     2 1.2116e+07    0   90 1.2199e+07 1.2116e+07  0.68%     -    1s
H   31    38                    1.219615e+07 1.2117e+07  0.65%   120    1s
H  112   159                    1.219591e+07 1.2117e+07  0.65%  97.9    2s
H  255   342                    1.219562e+07 1.2117e+07  0.65%  83.8    2s
H  412   409                    1.219542e+07 1.2117e+07  0.65%  73.6    2s
H  675   572                    1.219525e+07 1.2117e+07  0.64%  70.0    3s
H  953   694                    1.219514e+07 1.2117e+07  0.64%  66.9    3s
  1411   912 1.2195e+07   11   90 1.2195e+07 1.2117e+07  0.64%  67.0    5s
H 1449   889                    1.219482e+07 1.2148e+07  0.39%  65.2    7s
H 1454   848             

In [7]:
model_1_result_df.to_csv('model_1_output.csv', index=False)

In [6]:
#trial output prints
"""
for i in cost_results:
    print(i)

for i in total_dist_results:
    print(i)
    
for i in alpha_options:
    print(i)
"""

'\nfor i in cost_results:\n    print(i)\n\nfor i in total_dist_results:\n    print(i)\n    \nfor i in alpha_options:\n    print(i)\n'

In [ ]:
#population assignment visualization
"""
for j in num_areas:
    for i in num_pods:
        if assign[i, j].x > 0:
            print("area:" + str(j) + '\t->\t' + "POD:" + str(i) + ' ' + str(assign[i, j].x))
"""

## model 2: baseline - minimizing maximum travel distance by an individual


In [14]:
# set up the model and running in loop

alpha_options = [0.01, 0.02, 0.04, 0.06, 0.08, 
                 0.10, 0.20, 0.30, 0.40, 0.50, 
                 0.60, 0.70, 0.80, 0.90, 0.92, 
                 0.94, 0.96, 0.98, 0.99] # weight for cost

#empty dataframe to collect the output to create CSV of the model output
model_2_result_df = pd.DataFrame()


#POD capacity as percentage of the total population:
POD_cap_percent_pop_list = [0.02, 0.025, 0.03, 0.05, 0.07]


for POD_cap_percent_pop in POD_cap_percent_pop_list:
    
    maxCapacity = sumPopulation * POD_cap_percent_pop

    cost_results = list()

    max_dist_results = list()
    
    total_dist_results = list()
    
    cost_norm = 15*10**6 # normalization factor: average cost from model-1 optiomization at max POD capacity of 3%

    dist_norm = 12 # normalization factor: average of the distances between the PODs and the zip codes

    for alpha in alpha_options:

        m2 = Model("facility")

        built = m2.addVars(num_pods, vtype=GRB.BINARY) #1 if built; X_j, 47
        assign = m2.addVars(num_pods, num_areas, lb = 0.0) #F_ij, 47 * 98
        assign_1_0 = m2.addVars(num_pods, num_areas, lb = 0.0, vtype=GRB.INTEGER)
        capacity = m2.addVars(num_pods, lb = 0.0, vtype=GRB.INTEGER)
        max_travel_dist = m2.addVar(name = "max_travel_dist", lb = 0.0)
        total_cost = m2.addVar(name = "total_cost", lb = 0.0)

        # set up objective

        #max_travel_dist = max(distance[pod][area] * built[pod] for area in num_areas for pod in num_pods)

        m2.setObjective((1-alpha) * max_travel_dist / dist_norm + 
                        alpha * total_cost / cost_norm, GRB.MINIMIZE)

        # add constraints

        # number of pods constraint
        # m.addConstr(sum(built[j] for j in num_pods) == 20)

        # no assignment, travel should appear without a POD built
        for area in num_areas:
            for pod in num_pods:
                m2.addConstr(assign[pod, area] <= built[pod])

        # capacity of a pod should be zero for the locations where the POD is not built
        bigm = sumPopulation

        for pod in num_pods:
            m2.addConstr(capacity[pod] <= built[pod] * bigm)


        # upper bound for assigning fraction of population to a POD:
        for area in num_areas:
            for pod in num_pods:
                m2.addConstr(assign[pod, area] <= 1)


        # Capacity limit for all PODs: capacity of all PODs should be less than maxCapacity
        for pod in num_pods:
            m2.addConstr(capacity[pod] <= maxCapacity)


        # Capacity constraint: population assigned to a pod should be smaller than the POD capacity
        m2.addConstrs(gp.quicksum(int(population.iloc[area]['population']) * assign[pod, area] for area in num_areas)
                         <= capacity[pod] for pod in num_pods)


        # minimum coverage constraint
        m2.addConstr(minPercent * sumPopulation <= gp.quicksum(capacity[pod] for pod in num_pods))




        # population coverage constraint applied on each zip code
        for area in num_areas:
            m2.addConstr(gp.quicksum(assign[pod, area] for pod in num_pods) >= minPercent)

        """ to apply the above constraint on overall population instead of separately on each zipcode -
        m2.addConstr(gp.quicksum(gp.quicksum(assign[pod, area] 
                                             for pod in num_pods) * 
                                 int(population.iloc[area]['population']) 
                                 for area in num_areas) >= minPercent * sumPopulation)
        """


        # one person gets assigned at max only once
        for area in num_areas:
            m2.addConstr(gp.quicksum(assign[pod, area] for pod in num_pods) <= 1)


        # assign_1_0
        for area in num_areas:
            for pod in num_pods:
                m2.addConstr(assign_1_0[pod, area] >= assign[pod, area])

        # maximum travel distance for an individual 
        for area in num_areas:
            for pod in num_pods:
                m2.addConstr(assign_1_0[pod,area] * distance[pod][area] <= max_travel_dist)


        m2.addConstr(gp.quicksum((h1 * capacity[pod] + h2 * T1 + h3 * T2 + h4 * T3) * built[pod] +
                                 (h5 * capacity[pod] + h6 * T1 + h7 * T2 + h8 * T3) * built[pod] for pod in num_pods) == total_cost)

        m2.addConstr(total_cost <= max_budget)

        m2.Params.TimeLimit = 60
        # Solve
        m2.optimize()

        assign_total = {} # dict of pod: assigned population
        for pod in num_pods:
            total_pop = 0
            for area in num_areas:
                total_pop += population.iloc[area]['population'] * assign[pod,area].X
            assign_total[pod] = total_pop


        sum(assign_total.values())/sumPopulation


        # total cost
        print(f"Total cost: {total_cost.X}")
        cost_results.append(total_cost.X)


        # max distance travelled
        print(f"Maximum distance travelled: {max_travel_dist.X}")
        max_dist_results.append(max_travel_dist.X)


        #travel distance:
        total_dist_opt = 0 # dict of pod: assigned population
        for pod in num_pods:
            for area in num_areas:
                total_dist_opt += (assign[pod,area].X * 
                                   distance[pod][area] * 
                                   int(population.iloc[area]['population']))

        #total distance travelled
        print(f"Total distance travelled: {total_dist_opt}")
        total_dist_results.append(total_dist_opt)


        
    # creating dataframe for the csv output from the model

    cost_column_name = 'Cost_at_pod_cap_' + str(POD_cap_percent_pop*100) + '_percent'
    max_dist_column_name = 'Max_Dist_at_pod_cap_' + str(POD_cap_percent_pop*100) + '_percent'
    tot_dist_column_name = 'Total_Dist_at_pod_cap_' + str(POD_cap_percent_pop*100) + '_percent'

    model_2_result_df['Alpha'] = alpha_options
    model_2_result_df[str(cost_column_name)] = cost_results
    model_2_result_df[str(max_dist_column_name)] = max_dist_results
    model_2_result_df[str(tot_dist_column_name)] = total_dist_results



Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 18763 rows, 9308 columns and 46296 nonzeros
Model fingerprint: 0xb80c036a
Model has 1 quadratic constraint
Variable types: 4608 continuous, 4700 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [7e-10, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.11s
Presolved: 14251 rows, 9355 columns, 46614 nonzeros
Variable types: 4607 continuous, 4748 integer (4653 binary)

Root relaxation: objective 3.788219e-02, 15065 iterations, 0.29 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

     0     0    0.20002    0  324    0.83341    0.20002  76.0%     -    6s
     0     0    0.20009    0  298    0.83341    0.20009  76.0%     -    6s
     0     0    0.35316    0   65    0.83341    0.35316  57.6%     -    7s
     0     0    0.39487    0   47    0.83341    0.39487  52.6%     -    8s
     0     0    0.43579    0   70    0.83341    0.43579  47.7%     -    8s
H    0     0                       0.8278789    0.43579  47.4%     -    8s
     0     0    0.43579    0   73    0.82788    0.43579  47.4%     -    8s
H    0     0                       0.8182789    0.43579  46.7%     -    8s
     0     0    0.43612    0   39    0.81828    0.43612  46.7%     -    8s
H    0     0                       0.8098789    0.43612  46.1%     -    8s
     0     0    0.43651    0   31    0.80988    0.43651  46.1%     -    8s
     0     0    0.43651    0   32    0.80988    0.43651  46.1%     -    8s
     0     0    0.43713    0   87    0.80988    0.43713  46.0%     -    8s
H    0     0             


Cutting planes:
  Gomory: 3
  Flow cover: 2

Explored 235 nodes (57394 simplex iterations) in 5.62 seconds
Thread count was 12 (of 12 available processors)

Solution count 10: 0.465609 0.465609 0.465872 ... 1.39677

Optimal solution found (tolerance 1.00e-04)
Best objective 4.656093600000e-01, best bound 4.656093600000e-01, gap 0.0000%
Total cost: 16677380.0
Maximum distance travelled: 4.912999999999999
Total distance travelled: 3289939.7915999303
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 18763 rows, 9308 columns and 46296 nonzeros
Model fingerprint: 0xa5984142
Model has 1 quadratic constraint
Variable types: 4608 continuous, 4700 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objecti

H    0     0                       0.5480131    0.54655  0.27%     -    8s
     0     0    0.54662    0   45    0.54801    0.54662  0.25%     -    8s
     0     0    0.54666    0   29    0.54801    0.54666  0.25%     -    8s
     0     0    0.54666    0   30    0.54801    0.54666  0.25%     -    8s
     0     0    0.54771    0    5    0.54801    0.54771  0.05%     -    8s
     0     0    0.54771    0    5    0.54801    0.54771  0.05%     -    8s
     0     0     cutoff    0         0.54801    0.54801  0.00%     -    8s

Cutting planes:
  Gomory: 3
  MIR: 3

Explored 1 nodes (96236 simplex iterations) in 8.63 seconds
Thread count was 12 (of 12 available processors)

Solution count 10: 0.548013 0.548013 0.548013 ... 0.918212

Optimal solution found (tolerance 1.00e-04)
Best objective 5.480130666667e-01, best bound 5.480130666667e-01, gap 0.0000%
Total cost: 16535980.0
Maximum distance travelled: 4.913000000000001
Total distance travelled: 3138425.579
Changed value of parameter TimeLimit 

     0     0    0.68209    0    9    0.74943    0.68209  8.99%     -   11s
     0     0    0.68302    0   32    0.74943    0.68302  8.86%     -   11s
     0     0    0.68426    0   32    0.74943    0.68426  8.70%     -   11s
H    0     0                       0.7481595    0.68426  8.54%     -   11s
     0     0    0.68426    0   33    0.74816    0.68426  8.54%     -   11s
     0     0    0.68524    0   40    0.74816    0.68524  8.41%     -   11s
     0     0    0.68661    0   40    0.74816    0.68661  8.23%     -   11s
H    0     0                       0.7102429    0.68679  3.30%     -   11s
     0     0    0.68694    0   38    0.71024    0.68694  3.28%     -   11s
     0     0    0.68872    0   32    0.71024    0.68872  3.03%     -   11s
H    0     0                       0.6969816    0.68872  1.18%     -   11s
     0     0    0.69183    0   30    0.69698    0.69183  0.74%     -   11s
     0     0    0.69185    0   30    0.69698    0.69185  0.74%     -   11s
H    0     0             

...

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 18763 rows, 9308 columns and 46296 nonzeros
Model fingerprint: 0x39428621
Model has 1 quadratic constraint
Variable types: 4608 continuous, 4700 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [2e-08, 6e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.12s
Presolved: 14251 rows, 9355 columns, 46614 nonzeros
Variable types: 4607 continuous, 4748 integer (4653 binary)
Found heuristic solution: objective 2.5379581

Root relaxation: objective 5.231824e-02, 25004 iterations, 3.29 seconds
Total elapsed time = 5.06s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

     0     0    0.64279    0   24    0.65478    0.64279  1.83%     -   19s
H    0     0                       0.6537514    0.64279  1.68%     -   19s
     0     0    0.64714    0   24    0.65375    0.64714  1.01%     -   19s
H    0     0                       0.6471451    0.64714  0.00%     -   19s

Cutting planes:
  MIR: 7
  Flow cover: 2
  Relax-and-lift: 2

Explored 1 nodes (153521 simplex iterations) in 19.14 seconds
Thread count was 12 (of 12 available processors)

Solution count 10: 0.647145 0.653751 0.654778 ... 0.779172

Optimal solution found (tolerance 1.00e-04)
Best objective 6.471450666667e-01, best bound 6.471447733333e-01, gap 0.0000%
Total cost: 13238102.0
Maximum distance travelled: 4.941
Total distance travelled: 3148659.0270000007
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 18763 rows, 9308 columns and 46296 nonzeros
Model fingerprint: 0x4273b4dc
Model has 1 quadratic constraint
Variable types: 4608 continuous, 4700 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [5e-08, 2e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.13s
Presolved: 14251 rows, 9355 columns, 46614 nonzeros
Variable types: 4607 continuous, 4748 integer (4653 binary)
Found heuristic solution: objective 3.1060828

Root relaxation: objective 9.424074e-02, 15595 iterations, 1.46 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

H  816   256                       0.8037441    0.78616  2.19%   144   19s
  1113   285    0.78616   19  117    0.80374    0.78616  2.19%   127   20s
  2143   485    0.78616  111  122    0.80374    0.78616  2.19%   131   25s
* 2397   519             127       0.8034308    0.78616  2.15%   128   25s
* 2765   604             115       0.8018233    0.78616  1.95%   122   26s
  2838   549    0.78988  145  653    0.80182    0.78754  1.78%   119   30s

Cutting planes:
  Gomory: 48
  MIR: 6
  Flow cover: 20
  RLT: 19
  Relax-and-lift: 2

Explored 2838 nodes (452060 simplex iterations) in 30.74 seconds
Thread count was 12 (of 12 available processors)

Solution count 10: 0.801823 0.803431 0.803744 ... 0.854704

Optimal solution found (tolerance 1.00e-04)
Best objective 8.018233066667e-01, best bound 8.018233066667e-01, gap 0.0000%
Total cost: 12435380.0
Maximum distance travelled: 5.8679999999999986
Total distance travelled: 3489992.7719999985
Changed value of parameter TimeLimit to 60.0
   Pre

  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.12s
Presolved: 14251 rows, 9355 columns, 46614 nonzeros
Variable types: 4607 continuous, 4748 integer (4653 binary)
Found heuristic solution: objective 3.3106077

Root relaxation: objective 1.065456e-01, 9625 iterations, 0.96 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.10655    0 1623    3.31061    0.10655  96.8%     -    2s
H    0     0                       0.9284768    0.10655  88.5%     -    2s
     0     0    0.19546    0 1313    0.92848    0.19546  78.9%     -    4s
H    0     0                       0.8338067    0.19546  76.6%     -    4s
H    0     0                       0.8276432    0.19546  76.4%     -    6s
H    0     0                       0.8208950    0.19546  76.2%     -    6s
     0     0    0.70723    0 1236    0.82090    0.70723  13.8%     -    

In [15]:
model_2_result_df.to_csv('model_2_output.csv', index=False)

In [16]:
#trial output prints
"""
for i in cost_results:
    print(i)
    
for i in max_dist_results:
    print(i)
    
for i in total_dist_results:
    print(i)
    
for i in alpha_options:
    print(i)
"""

'\nfor i in cost_results:\n    print(i)\n    \nfor i in max_dist_results:\n    print(i)\n    \nfor i in total_dist_results:\n    print(i)\n    \nfor i in alpha_options:\n    print(i)\n'

In [17]:
#population assignment visualization
"""
for j in num_areas:
    for i in num_pods:
        if assign[i, j].x > 0:
            print("area:" + str(j) + '->' + "POD:" + str(i) + ' ' + str(assign[i, j].x))
"""

'\nfor j in num_areas:\n    for i in num_pods:\n        if assign[i, j].x > 0:\n            print("area:" + str(j) + \'->\' + "POD:" + str(i) + \' \' + str(assign[i, j].x))\n'